General Imports


In [52]:
# numpy imports
import numpy as np

# Sklearn imports
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, default_collate, TensorDataset

# scipy
from scipy.io import arff

# pandas
import pandas as pd

In [53]:
# Setup pytorch device
# I have a GPU (RTX 3080) so I want to take advantage of that

use_cuda = True # simple way to disable in case there are issues

if torch.cuda.is_available() and use_cuda:
    my_device = torch.device("cuda")
else:
    my_device = torch.device("cpu")
print('Device: {}'.format(my_device))

torch.manual_seed(0)

Device: cuda


In [54]:
!nvidia-smi

Wed Nov 16 16:42:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:09:00.0  On |                  N/A |
| 54%   59C    P2   118W / 400W |   3031MiB / 12288MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [55]:

data = pd.read_csv('datasets/NSL-KDD/KDDTrain+_with_header.txt', header=0)
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [56]:
mappings = ['protocol_type', 'service', 'flag', 'class']
column_mappings = {}
for to_map in mappings:
    data.get(to_map).unique()
    unique_vals = data.get(to_map).unique()
    data[to_map].replace(to_replace=unique_vals,
            value= list(range(len(unique_vals))),
            inplace=True)
    
    # set column mappings
    column_mappings[to_map] = {}
    for intMaping, key in enumerate(unique_vals):
        column_mappings[to_map][intMaping] = key

In [57]:
class LinearNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layer, output_dim, device):
        super().__init__()
        self.device = device
        self.hidden_layer = hidden_layer
        self.output_dim = output_dim
        self.linear1 = nn.Linear(input_dim, hidden_layer).to(device)
        self.activation = nn.ReLU().to(device)
        self.linear2 = nn.Linear(hidden_layer, output_dim).to(device)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        return x.to(self.device)


In [58]:
target = data['class']
del data['class']
train = TensorDataset(torch.Tensor(np.array(data)), torch.Tensor(np.array(target)).type(torch.LongTensor))
train_loader = DataLoader(train, batch_size = 50, shuffle = True, collate_fn=lambda x: tuple(x_.to(my_device) for x_ in default_collate(x)))

In [59]:

lr = 0.001
clf = LinearNetwork(41, 50, 2, my_device)
clf.to(my_device)

clfLinearCriterion = nn.CrossEntropyLoss().to(my_device)
clfLinearOptimizer = torch.optim.SGD(clf.parameters(), lr=lr)

In [60]:
# train all the classifiers...
epochs = 3000
epoch_display_rate = 5
clf.train()
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    # zero grad to remove previous gradient values
    clfLinearOptimizer.zero_grad()
    # forward propagation
    outputs = clf(inputs)
    loss = clfLinearCriterion(outputs, labels)
    # backward propagation
    loss.backward()
    # optimize
    clfLinearOptimizer.step()
    running_loss += loss.item()
  # display statistics
  if (epoch % epoch_display_rate == 0):
    print(f'classifier: Epoch: {epoch:5d} loss: {running_loss / 2000:.5f}')

classifier: Epoch:     0 loss: 83.70534
classifier: Epoch:     5 loss: 0.23327
classifier: Epoch:    10 loss: 0.22101
classifier: Epoch:    15 loss: 0.21779
classifier: Epoch:    20 loss: 0.21220
classifier: Epoch:    25 loss: 0.23662
classifier: Epoch:    30 loss: 0.22653
classifier: Epoch:    35 loss: 0.22361
classifier: Epoch:    40 loss: 0.22590
classifier: Epoch:    45 loss: 0.25269
classifier: Epoch:    50 loss: 0.23094
classifier: Epoch:    55 loss: 0.22612
classifier: Epoch:    60 loss: 0.22737
classifier: Epoch:    65 loss: 0.25090
classifier: Epoch:    70 loss: 0.29684
classifier: Epoch:    75 loss: 0.29003
classifier: Epoch:    80 loss: 0.20782
classifier: Epoch:    85 loss: 0.29152
classifier: Epoch:    90 loss: 0.29633
classifier: Epoch:    95 loss: 0.29112
classifier: Epoch:   100 loss: 0.28749
classifier: Epoch:   105 loss: 0.28721
classifier: Epoch:   110 loss: 0.28703
classifier: Epoch:   115 loss: 0.28670
classifier: Epoch:   120 loss: 0.21056
classifier: Epoch:   125

In [61]:
print("Losses for classifier")

correct, total = 0, 0
# no.grad context to not have to calculate the gradiants
with torch.no_grad():
    for ___, data in enumerate(train_loader):
        inputs, labels = data
        outputs = clf(inputs)
        ____, predicted = torch.max(outputs.data, 1)

        # update results
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'\n\nAccuracy on test points: {100 * correct // total}%')

Losses for classifier


Accuracy on test points: 95%
